In [1]:
import pandas as pd
from itertools import combinations
import math
import os
import csv
import time

/var/folders/gp/vqql8_v1253_d8bwfqk_0gb40000gn/T/ipykernel_2080/158659454.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
cargo_capacity_psv = 100
psv_speed = 7 #7 for strøm, 10 på resten
max_platforms_in_one_voyage = 7

## Reads input data

In [3]:
demand = pd.read_csv('clustering/output_platforms_demand.csv', header=0, delimiter=';')
distances = pd.read_csv('clustering/output_distance_matrix_kmeans.csv', header=0, delimiter=';', index_col='from/to')

platforms_demand = dict(zip(demand['platform'], demand['avg_q'].replace(',', '.').astype(float)))
platforms_d = ['DUS'] + demand['platform'].tolist() + ['DUS']  # Add 'DUS' as start and end platforms

## Algorithm for generating the shortest routes

In [4]:
shortest_routes_dict = {}

def generate_routes(demand, distances):
    cargo_capacity = cargo_capacity_psv
    max_platforms = max_platforms_in_one_voyage + 2

    def dp(platform, cargo_remaining, route, visited):
        if cargo_remaining < 0:
            return
        if len(route) > max_platforms:
            return
        if platform == 'DUS' and len(route) > 2:
            total_demand = sum(platforms_demand[p] for p in route[1:-1])
            if total_demand <= cargo_capacity:
                key = tuple(sorted(set(route)))
                total_distance = sum(distances.loc[route[i], route[i+1]] for i in range(len(route)-1))
                if key not in shortest_routes_dict or total_distance < shortest_routes_dict[key][1]:
                    duration_sailing = round((total_distance / psv_speed), 2)
                    duration_lossing = round(((total_demand * 1.389) / psv_speed), 2)
                    duration_sailing = round(duration_sailing, 2)
                    duration_lossing = round(duration_lossing, 2)
                    shortest_routes_dict[key] = (route, total_distance, total_demand, duration_sailing, duration_lossing)
            return

        # Check if the current route is dominated
        current_distance = sum(distances.loc[route[i], route[i+1]] for i in range(len(route)-1))
        current_demand = sum(platforms_demand[p] for p in route[1:-1])
        
        # Check for dominance in existing routes
        for key, (existing_route, existing_distance, existing_demand, _, _) in shortest_routes_dict.items():
            if set(existing_route[1:-1]) == set(route[1:-1]) and existing_demand >= current_demand and existing_distance <= current_distance:
                return
        
        # Check for dominance in subsequent routes starting with the same platforms
        for existing_route in dominated_routes:
            if set(existing_route[1:-1]) == set(route[1:-1]) and existing_demand >= current_demand and existing_distance <= current_distance:
                return

        for next_platform in platforms_demand.keys():
            if next_platform != platform and next_platform not in visited:
                try:
                    distance_to_next = distances.loc[platform, next_platform]
                    new_cargo_remaining = cargo_remaining - platforms_demand[next_platform]
                    dp(next_platform, new_cargo_remaining, route + [next_platform], visited.union({next_platform}))
                except KeyError:
                    print("KeyError occurred for platform:", next_platform)
                    continue

    for r in range(3, min(len(platforms_demand) + 3, max_platforms + 3)):
        for route_combination in combinations(platforms_demand.keys(), r - 2):
            route = ['DUS'] + list(route_combination) + ['DUS']
            dp('DUS', cargo_capacity, route, {'DUS'})

            # Add the current route to the dominated routes set
            dominated_routes.add(tuple(route))

    return shortest_routes_dict

# Keep track of dominated routes to skip future routes starting with them
dominated_routes = set()
shortest_routes_dict = generate_routes(demand, distances)

for route, distance, demand, duration_sailing, duration_lossing in shortest_routes_dict.values():
    print(f"Shortest Route: {route}, Total Distance: {round(distance,2)}, Total Demand: {demand}, Duration sailing: {duration_sailing}, Duration lossing: {duration_lossing}")

print(len(shortest_routes_dict))

Shortest Route: ['DUS', 'DAB', 'DUS'], Total Distance: 185.84, Total Demand: 40.0, Duration sailing: 26.55, Duration lossing: 7.94
Shortest Route: ['DUS', 'DRA', 'DUS'], Total Distance: 205.36, Total Demand: 14.0, Duration sailing: 29.34, Duration lossing: 2.78
Shortest Route: ['DUS', 'DSA', 'DUS'], Total Distance: 164.4, Total Demand: 29.0, Duration sailing: 23.49, Duration lossing: 5.75
Shortest Route: ['DUS', 'GKR', 'DUS'], Total Distance: 234.68, Total Demand: 18.0, Duration sailing: 33.53, Duration lossing: 3.57
Shortest Route: ['DUS', 'GUD', 'DUS'], Total Distance: 226.76, Total Demand: 18.0, Duration sailing: 32.39, Duration lossing: 3.57
Shortest Route: ['DUS', 'JSF', 'DUS'], Total Distance: 176.18, Total Demand: 40.0, Duration sailing: 25.17, Duration lossing: 7.94
Shortest Route: ['DUS', 'GRA', 'DUS'], Total Distance: 180.44, Total Demand: 14.0, Duration sailing: 25.78, Duration lossing: 2.78
Shortest Route: ['DUS', 'HDA', 'DUS'], Total Distance: 206.82, Total Demand: 18.0, D

## Saves the shortest routes left to csv files in the generated_datafiles folder. In total 858 routes

In [6]:
routes_only = [route for route, _, _, _, _ in shortest_routes_dict.values()]
distances_only = [distance for _, distance, _, _, _ in shortest_routes_dict.values()]
demand_only = [demand for _, _, demand, _, _ in shortest_routes_dict.values()]
duration_sailing = [duration_sailing for _, _, _, duration_sailing, _ in shortest_routes_dict.values()]
duration_lossing = [duration_lossing for _, _, _, _, duration_lossing in shortest_routes_dict.values()]

df_distances = pd.DataFrame({'Distance': distances_only})
df_demand = pd.DataFrame({'Demand': demand_only})
df_duration_sailing = pd.DataFrame({'Duration (hours)': duration_sailing})
df_duration_lossing = pd.DataFrame({'Duration (hours)': duration_lossing})

df_distances.to_csv('generated_datafiles/distances.csv', index=False)
df_demand.to_csv('generated_datafiles/demand.csv', index=False)
df_duration_sailing.to_csv('generated_datafiles/duration_sailing.csv', index=False)
df_duration_lossing.to_csv('generated_datafiles/duration_lossing.csv', index=False)
route_file = "generated_datafiles/routes.csv"


def write_to_csv(filename, data):
    with open(filename, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(data)

write_to_csv(route_file, routes_only)